In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.datasets import load_sample_image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.ndimage import shift
import matplotlib.pyplot as plt

In [3]:
"""aug = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    brightness_range=[0.7, 1.3],  
    fill_mode="constant",  
    cval=0  
)
"""

'aug = tf.keras.preprocessing.image.ImageDataGenerator(\n    width_shift_range=0.2,  \n    height_shift_range=0.2,  \n    brightness_range=[0.7, 1.3],  \n    fill_mode="constant",  \n    cval=0  \n)\n'

In [4]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float16") / 255.0
x_test = x_test.astype("float16") / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)



In [5]:
"""augmented_images = []
augmented_labels = []
for image, label in zip(x_train, y_train):
    image = np.expand_dims(image, axis=0)  
    augment_iter = aug.flow(image, batch_size=1) 
    for _ in range(2):
        aug_image = next(augment_iter)[0]
        augmented_images.append(aug_image)
        augmented_labels.append(label)


augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
x_train_augmented = np.concatenate([x_train, augmented_images])
y_train_augmented = np.concatenate([y_train, augmented_labels])

print(x_train.shape)
print(x_train_augmented.shape)
"""

'augmented_images = []\naugmented_labels = []\nfor image, label in zip(x_train, y_train):\n    image = np.expand_dims(image, axis=0)  \n    augment_iter = aug.flow(image, batch_size=1) \n    for _ in range(2):\n        aug_image = next(augment_iter)[0]\n        augmented_images.append(aug_image)\n        augmented_labels.append(label)\n\n\naugmented_images = np.array(augmented_images)\naugmented_labels = np.array(augmented_labels)\nx_train_augmented = np.concatenate([x_train, augmented_images])\ny_train_augmented = np.concatenate([y_train, augmented_labels])\n\nprint(x_train.shape)\nprint(x_train_augmented.shape)\n'

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=60)

In [7]:
x_train_flat = x_train.reshape(-1, 28 * 28)
x_val_flat = x_val.reshape(-1, 28 * 28)
x_test_flat = x_test.reshape(-1, 28 * 28)


In [8]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_val_scaled = scaler.transform(x_val_flat)
x_test_scaled = scaler.transform(x_test_flat)

In [9]:
x_train = x_train_flat.reshape(-1, 28, 28, 1)
x_val = x_val_flat.reshape(-1, 28, 28, 1)
x_test = x_test_flat.reshape(-1, 28, 28, 1)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 5, padding="same", input_shape=[28, 28, 1]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(128, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(128, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Conv2D(256, 3, padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(10, activation="softmax")
])


C:\Users\marce\anaconda4\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 128)            │             

 Total params: 1,416,394 (5.40 MB)

 Trainable params: 1,414,346 (5.40 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [12]:
def lr_schedule(epoch):
    if epoch < 3:
        return 0.05
    elif epoch < 7:
        return 0.01
    elif epoch < 12:
        return 0.001
    elif epoch < 16:
        return 0.0005
    else:
        return 0.0001


In [13]:
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy")
]

In [14]:
optimizer = tf.keras.optimizers.Nadam()
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [15]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=callbacks
)


Epoch 1: LearningRateScheduler setting learning rate to 0.05.
Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 507s 325ms/step - accuracy: 0.8119 - loss: 0.6161 - val_accuracy: 0.8592 - val_loss: 0.5592 - learning_rate: 0.0500

Epoch 2: LearningRateScheduler setting learning rate to 0.05.
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 510s 340ms/step - accuracy: 0.9334 - loss: 0.2628 - val_accuracy: 0.9480 - val_loss: 0.2747 - learning_rate: 0.0500

Epoch 3: LearningRateScheduler setting learning rate to 0.05.
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 518s 345ms/step - accuracy: 0.9463 - loss: 0.2165 - val_accuracy: 0.9542 - val_loss: 0.2174 - learning_rate: 0.0500

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 525s 350ms/step - accuracy: 0.9706 - loss: 0.1197 - val_accuracy: 0.9892 - val_loss: 0.0443 - learning_rate: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 515s 343ms/

In [16]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 30s 76ms/step - accuracy: 0.9954 - loss: 0.0232
Test Loss: 0.02198329195380211
Test Accuracy: 0.9961000084877014


In [17]:
print(history.history)

{'accuracy': [0.8888333439826965, 0.9366458058357239, 0.9456666707992554, 0.97677081823349, 0.9823958277702332, 0.9827083349227905, 0.9839791655540466, 0.9887708425521851, 0.9912499785423279, 0.9915416836738586, 0.9917500019073486, 0.9922916889190674, 0.9925416707992554, 0.9940624833106995, 0.9932708144187927, 0.9932708144187927, 0.9933124780654907, 0.9940000176429749, 0.9939374923706055, 0.9938958287239075], 'loss': [0.39548197388648987, 0.2474471479654312, 0.21589766442775726, 0.0951339527964592, 0.07281212508678436, 0.0696907490491867, 0.06162744760513306, 0.044482264667749405, 0.03603045269846916, 0.033519282937049866, 0.03259425237774849, 0.029025793075561523, 0.02823888696730137, 0.022733958438038826, 0.026962630450725555, 0.026303846389055252, 0.024272503331303596, 0.021637514233589172, 0.02243337593972683, 0.023093855008482933], 'val_accuracy': [0.85916668176651, 0.9480000138282776, 0.9541666507720947, 0.9891666769981384, 0.9921666383743286, 0.9912499785423279, 0.98975002765655